## Key point of Debevec is to reconstrcut the camera's response curve to incoming light. 
And then, a simple range compression is applied. This later part is more relevant to my problem. In that regard, Debevec is not very useful... Or is it? 


## Questions
* Can I assume **UNIFORM** response function of telescopes? -- probably, Yes.

# OpenCV HDR examples
## Debevec, Robertson and ...?

In [1]:
import numpy as np
import cv2 as cv

img_fn_base = "/home/hoseung/Work/tonemap/astroBF/notebooks/natural/StLouisArchMultExpEV"
fn_suffix = ["+4.09.jpeg", "+1.51.jpeg", "-1.82.jpeg" ,"-4.72.jpeg"]
img_list = [cv.imread(img_fn_base+fn) for fn in fn_suffix]
exposure_times = np.array([15.0, 2.5, 0.25, 0.0333], dtype=np.float32)

# Merge muliti-exposure images into one HDR image 
This step won't be necessary for astronomical images. 
So, let's just use a pre-built method.


Available TMOs in OpenCV are:  
cv::TonemapDrago, cv::TonemapMantiuk and cv::TonemapReinhard

In [10]:
# Merge exposures to HDR image
merge_debevec = cv.createMergeDebevec()
hdr_debevec = merge_debevec.process(img_list, times=exposure_times.copy())

merge_robertson = cv.createMergeRobertson()
hdr_robertson = merge_robertson.process(img_list, times=exposure_times.copy())

# Estimate camera response function (CRF)
cal_debevec = cv.createCalibrateDebevec()
crf_debevec = cal_debevec.process(img_list, times=exposure_times)
hdr_debevec = merge_debevec.process(img_list, times=exposure_times.copy(), response=crf_debevec.copy())
cal_robertson = cv.createCalibrateRobertson()
crf_robertson = cal_robertson.process(img_list, times=exposure_times)
hdr_robertson = merge_robertson.process(img_list, times=exposure_times.copy(), response=crf_robertson.copy())

# Tonemap HDR image
tonemap1 = cv.createTonemap(gamma=2.2)
res_debevec = tonemap1.process(hdr_debevec.copy())

res_robertson = tonemap1.process(hdr_robertson.copy())

# Exposure fusion using Mertens
merge_mertens = cv.createMergeMertens()
res_mertens = merge_mertens.process(img_list)

# Convert datatype to 8-bit and save
res_debevec_8bit = np.clip(res_debevec*255, 0, 255).astype('uint8')
res_robertson_8bit = np.clip(res_robertson*255, 0, 255).astype('uint8')
res_mertens_8bit = np.clip(res_mertens*255, 0, 255).astype('uint8')
cv.imwrite("ldr_debevec_cali.jpg", res_debevec_8bit)
cv.imwrite("ldr_robertson_cali.jpg", res_robertson_8bit)
cv.imwrite("fusion_mertens.jpg", res_mertens_8bit)

# Custom implementation 

of Ashi..., Fattai (Gradient Domain High Dynamic Range Compression), and..?

No Devebec. It's just irrelevant